# 30x30

## Mapping of habitats in protected areas

*Living Land Management* is a Monmouthshire-based project that is using data acquired by spaceborne and airborne sensors to assess how we use the land currently and consider options for future use that enhances business but concurrently supports local food production, restoration of environment (e.g., carbon stocks, biodiversity) and increases resilience to climate change.  

The project is a novel and unique collaboration between Monmouthshire County, Aberystwyth University, Dwr Cymru Welsh Water and Natural Resources Wales that links to *Living Wales*, a national initiative that is giving free and open access to remotely sendsed data and products to support wise use of the Welsh landcdape and a better collective outlook for current and future generatiouns.

## Information about your land holding.

This brochure extracts information on your landholding for different years from the newly developed Welsh Data Cube (WDC), which houses all satellite data acquired over Wales since 2018 and derived products with these including land cover, broad habitats and water/moisture persistence.  

**Land Cover** is the physical and biological cover of the land surface and includes vegetation (managed or semi-natural), water and bare surfaces.  The land cover maps generated through Living Land Management use the legends of the United Nation's Food and Agriculture Organisation (FAO) Land Cover Classification System (LCCS).

**Habitats** represent the natural environments in which individual or groups of plant or animal species lives.  The habitat maps are generated from satellite data and are based on Wales' Phase 1 Habitat Taxonomy.

The **water/moisture persistence** is obtained from time-series of radar data that are acquired almost every day over Wales and indicate relative frequency of wet conditions across the landscape.  

In [1]:
import datacube
import geopandas as gpd
import xarray as xr
import numpy as np
import os
import sys
#sys.path.append("../notebooks/wales_utils/data_cube_utilities")
#from display_tools import map_geom, rgb
sys.path.append("../Tools/wdc_tools")
from display_tools import map_geom, rgb
from wdc_datahandling import geopolygon_masking

from datacube.utils.geometry import Geometry, CRS
from ipyleaflet import GeoData

from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

In [2]:
def stat_summary(xarr, scheme):
    # Search habitat types in farm 
    farm_types = np.unique(xarr, return_counts=True)
    
    # Create dictionary of habitats with respective area in hectares
    habitat_stat_dict = {}
    for color, label in scheme.items():
        if((label[0] in farm_types[0]) & (label[0]!=0)):
            habitat_stat_dict[label[1]] = farm_types[1][list(farm_types[0]).index(label[0])]*100/10000 

    # Add total area
    habitat_stat_total_dict = {"TOTAL": round(sum(habitat_stat_dict.values()),2)}

    # Calculate percentage of each habitat
    for label, value in habitat_stat_dict.items():
        percentage = round(value/habitat_stat_total_dict["TOTAL"], 2)
        habitat_stat_dict[label] = [value, percentage]

    ## Python program to print the habitat details into table 
    print ("{:<54} {:<10} {:<20}".format("\033[1m" +'CATEGORY','HECTARE', "PERCENT"+"\033[0m"))
    for k, v in habitat_stat_dict.items():
        hect, perc = v
        print ("{:<50} {:<10} {:<20}".format(k, hect, perc))
    ## Python program to print TOTAL into table
    for k, v in habitat_stat_total_dict.items():
        print ("{:<50} {:<10}".format(k, v))


In [3]:
dc = datacube.Datacube(app='Area estimates')

In [5]:
product = "lc"

measurements = dc.list_measurements()
measurements.loc[product]

KeyError: 'lc'

In [7]:
# Step 1: Check if required variables are defined
if lat_range is None or lon_range is None or start_date_input.value is None or end_date_input.value is None:
    raise ValueError("Latitude, Longitude, or Time range is not defined")

print("Lat Range:", lat_range)
print("Lon Range:", lon_range)
print("Start Date:", start_date_input.value)
print("End Date:", end_date_input.value)

# Step 2: Construct the query
query = {
    "y": lat_range,
    "x": lon_range,
    "time": (start_date_input.value, end_date_input.value),
}

print("Query Parameters:", query)

# Step 3: Load DEA Land Cover data from the datacube
try:
    lc = dc.load(
        product="ga_ls_landcover_class_cyear_2",
        output_crs="EPSG:3577",
        measurements=[
            "level3",
            "lifeform",
            "vegetation_cover",
            "water_seasonality",
            "water_state",
            "intertidal",
            "water_persistence",
            "bare_gradation",
            "full_classification",
        ],
        resolution=(-25, 25),
        **query
    )
    print("Data loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Error loading data from the datacube: {e}")

# Step 4: Check if the dataset is loaded correctly
if lc is None or lc.geobox is None:
    raise ValueError("Failed to load the dataset. Please check the query parameters and ensure data is available for the specified region and time range.")   
    
# Step 5: Print geobox information
geobox = lc.geobox

geobox_info = f"""
Geobox Information:
-------------------
CRS: {geobox.crs}
Dimensions: {geobox.dimensions}
Resolution: {geobox.resolution}
Shape: {geobox.shape}
Extent: {geobox.extent}
"""

print(geobox_info)

# Step 6: Check loaded data dimensions and variables
print("Loaded data dimensions:", lc.dims)
print("Loaded data variables:", lc.data_vars)

NameError: name 'lat_range' is not defined

In [ ]:
product = "lw_habitats_lw"

measurements = dc.list_measurements()
measurements.loc[product]

Select an area:
- Wye_Valley_Wales
- Angelsey

In [ ]:
area_name = "Anglesey"
year = "2022"

In [ ]:
# Open and read the shapefiles
fields_path = '../uploads/Wales/Areas/Counties/'+area_name+'_Fields.shp'
fields_exists = os.path.isfile(fields_path)

boundary_path = '../uploads/Wales/Areas/Counties/'+area_name+'_Boundary.shp'
boundary_exists = os.path.isfile(boundary_path)

if fields_exists:
    Fields = gpd.read_file(fields_path)
if boundary_exists:
    Boundary = gpd.read_file(boundary_path)

In [ ]:
Boundary.head(3)

In [ ]:
# Transform shapefile boundaries into geographic data (and affect a style) 
geo_data = GeoData(geo_dataframe = Boundary.to_crs(epsg=4326),
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 
                          'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Boundary')

# map the geographic data on dynamic map
m = map_geom(geo_data)
m

In [ ]:
geom = Geometry(geom=Boundary.iloc[0].geometry, 
                         crs=CRS("epsg:27700"))
geom

In [ ]:
query = {'geopolygon': geom,
         'time': (year+"-01-01", year+"-12-31"),
         'output_crs': 'EPSG:27700',
         'resolution': (-10, 10)
        }

In [ ]:
# Load land cover data for our polygon and time period
lc_dataset = dc.load(product='lw_landcover_lw', **query)
lc_dataset_masked = geopolygon_masking(lc_dataset, geopolygon=geom)

In [ ]:
level3plus = ((lc_dataset_masked.level3.where(lc_dataset_masked.level3 == 112) + lc_dataset_masked.lifeform).fillna(0) + (
    lc_dataset_masked.level3.where(lc_dataset_masked.level3 != 112)).fillna(0))
lc_dataset["level3plus"] = level3plus

In [ ]:
# Level3plus colour scheme
level3plus_scheme = {"#FFFFFF": [0., "Not classified"],
                     "#D1E133": [111., "Cultivated or managed terrestrial vegetation"], 
                     "#007A02": [113., "Semi-natural terrestrial woody vegetation"], 
                     "#95c748": [114., "Semi-natural terrestrial herbaceous vegetation"],
                     "#4EEEE8": [123., "Cultivated or managed aquatic vegetation"],
                     "#02C077": [124., "Semi-natural aquatic vegetation"],
                     "#DA5C69": [215., "Artificial surface"],
                     "#F3AB69": [216., "Bare surface"],
                     "#4D9FDC": [220., "Water"]}

# Colour map
level3plus_cmap = ListedColormap(list(level3plus_scheme.keys()))
# Level3plus classes
# Define a normalization from values -> colors
level3plus_norm = colors.BoundaryNorm([value[0] for value in level3plus_scheme.values()], 9)


In [ ]:
# Plotting
lc_fig, ax = plt.subplots(figsize=(20, 10))

lc_plot = ax.imshow(lc_dataset.level3plus.isel(time=0),
                     cmap=level3plus_cmap,
                     norm=level3plus_norm,
                     extent=[lc_dataset.x.min().data, lc_dataset.x.max().data,
                             lc_dataset.y.min().data, lc_dataset.y.max().data])

if boundary_exists:
    Boundary.boundary.plot(ax=ax, ec='#e72323', linewidth=3)

if fields_exists:
    Fields.boundary.plot(ax=ax, ec='#220f46', linewidth=0.5)

patches = [Patch(color=color, label=label[1])
           for color, label in level3plus_scheme.items()]

ax.legend(handles=patches,
          bbox_to_anchor=(1.7, 0.7),
          facecolor="white")

#ax.set_axis_off()
plt.show()

In [ ]:
lc_fig.savefig('outputs/Land_cover_'+area_name+'_'+year+'.png')

In [ ]:
print('Level3 min', np.nanmax(lc_dataset["level3plus"]))

In [ ]:
f, ax = plt.subplots()
lc_dataset.level3plus.plot.hist(color="green")
ax.set(title="Distribution of Raster Cell Values in land cover",
       xlabel="Height (m)",
       ylabel="Number of Pixels")
plt.show()

In [ ]:
class_bins = [111,112,124,215,216,220]

In [ ]:
class_bins

In [ ]:
reclass = xr.apply_ufunc(np.digitize, lc_dataset.level3plus, class_bins)

In [ ]:
lc_dataset.assign(new=lc_dataset["level3plus"]*10)

#### Classify the area of each land cover in your farm

In [ ]:
stat_summary(lc_dataset.level3plus, level3plus_scheme)

In [ ]:
# Load habitat data for our polygon and time period
habitat_dataset = dc.load(product='lw_habitats_lw', **query)
habitat_dataset_masked = geopolygon_masking(habitat_dataset, geopolygon=geom)

In [ ]:
# Level3plus colour scheme
broadhabitat_scheme = {"#FFFFFF": [0., "Not classified"],
                       "#00C502": [1., "Broadleaved woodland"], 
                       "#006902": [2., "Needle-leaved woodland"],
                       "#CEF191": [3., "Semi-natural grassland"],
                       "#C91FCC": [4., "Heathland and Scrub"],
                       "#F2A008": [5., "Bracken"],
                       "#F8F8C9": [6., "Bog"],
                       "#177E88": [7., "Fen/Marsh/Swamp"],
                       "#FFFF00": [8., "Cultivated or managed vegetation"],
                       "#00DDA4": [9., "Coastal habitat"],
                       "#0E00ED": [10., "Open Water"],
                       "#908E8D": [11., "Natural Bare Surfaces"],
                       "#000000": [12., "Artificial Bare Surfaces"],
                       "#DAC654": [13., "Young trees/Felled/Coppice"],
                       "#5d994e": [14., "Woodland and scrub"]
                      }

# Habitat colour scheme
broadhabitat_cmap = ListedColormap(list(broadhabitat_scheme.keys()))
# Habitat classes
broadhabitat_norm = colors.BoundaryNorm([value[0] for value in broadhabitat_scheme.values()], 15)


In [ ]:
# Plotting
habitat_fig, ax = plt.subplots(figsize=(20, 10))

habitat_plot = ax.imshow(habitat_dataset_masked.broad.isel(time=0),
                     cmap=broadhabitat_cmap,
                     norm=broadhabitat_norm,
                     extent=[habitat_dataset.x.min().data, habitat_dataset.x.max().data,
                             habitat_dataset.y.min().data, habitat_dataset.y.max().data])

if boundary_exists:
    Boundary.boundary.plot(ax=ax, ec='#e72323', linewidth=3)
    
if fields_exists:
    Fields.boundary.plot(ax=ax, ec='#220f46', linewidth=0.5)


patches = [Patch(color=color, label=label[1])
           for color, label in broadhabitat_scheme.items()]

ax.legend(handles=patches,
          bbox_to_anchor=(1.6, 1.0),
          facecolor="white")

# ax.set_axis_off()
plt.show()

In [ ]:
habitat_fig.savefig('outputs/Broad_habitats_'+area_name+'_'+year+'.png')

#### Classify the area of each habitat in your farm

In [ ]:
stat_summary(habitat_dataset_masked.broad, broadhabitat_scheme)

In [ ]:
# Detailed habitat colour scheme
det_habitat_scheme = {"#FFFFFF": [0., "Not classified"],
                       "#45fb82": [3., "Semi-natural grassland (unclassified)"], 
                       "#577117": [4., "Juncus rushes"],
                       "#cef191":[5., "Molinia grassland"],
                       "#f8f096": [9., "Young plantations/felled/coppice"],
                       "#399c4f": [10., "Woodland and scrub (unclassified)"],
                       "#0ac72d": [12., "Broadleaved woodland"],
                       "#286b35": [16., "Needleleaved woodland"],
                       "#6d5742": [23., "Ulex dominated scrub"],
                       "#d7d236": [35., "Acid grassland"],
                       "#c5d833": [38., "Neutral grassland"],
                       "#c3c000": [41., "Calcareous grassland"],
                       "#fced13": [44., "Improved grassland"],
                       "#518388": [45., "Marsh/marshy grassland"],
                       "#f37e1c": [50., "Bracken"],
                       "#9c0f85": [58., "Dry dwarf shrub heath"],
                       "#e2a7ed": [61., "Wet dwarf shrub heath"],
                       "#df0f4a": [70., "Blanket sphagnum bog"],
                       "#f74428": [71., "Raised sphagnum bog"],
                       "#f7eeb6": [72., "Modified bog"],
                       "#6bdac2": [78., "Fen"],
                       "#493717": [85., "Peat - bare"],
                       "#4eb0e0": [86., "Swamp"],
                       "#013ff6": [90., "Open water"],
                       "#a5e1dc": [106., "Intertidal vegetation generic"],
                       "#3b4b61": [107., "Intertidal bare generic"],
                       "#4de5fc": [119., "Saltmarsh"],
                       "#ecb641": [128., "Sand dune"],
                       "#30aa87": [130., "Dune grassland"],
                       "#6e1ae3": [131., "Dune heath"],
                       "#7c3843": [132., "Dune scrub"],
                       "#7286bb": [134., "Maritime cliff and slope (unvegetated)"],
                       "#5a5860": [135., "Maritime cliff and slope (vegetated)"],
                       "#020b09": [142., "Natural rock exposure and waste"],
                       "#c6c3d3": [143., "Inland cliff"],
                       "#e7ebeb": [155., "Quarry"],
                       "#fb9a71": [159., "Arable crops"],
                       "#ddf0f1": [200., "Artificial bare surfaces"],
                       "#a0ada9": [201., "Natural bare surfaces"],
                       "#8cbd77": [202., "Semi-natural herbaceous vegetation (unclassified)"]
                      }

# Habitat colour scheme
det_habitat_cmap = ListedColormap(list(det_habitat_scheme.keys()))
# Habitat classes
det_habitat_norm = colors.BoundaryNorm([value[0] for value in det_habitat_scheme.values()], 40)

In [ ]:
# Plotting
habitat_fig, ax = plt.subplots(figsize=(20, 10))

habitat_plot = ax.imshow(habitat_dataset_masked.detailed.isel(time=0),
                     cmap=det_habitat_cmap,
                     norm=det_habitat_norm,
                     extent=[habitat_dataset.x.min().data, habitat_dataset.x.max().data,
                             habitat_dataset.y.min().data, habitat_dataset.y.max().data])

if boundary_exists:
    Boundary.boundary.plot(ax=ax, ec='#e72323', linewidth=3)
    
if fields_exists:
    Fields.boundary.plot(ax=ax, ec='#220f46', linewidth=0.5)


patches = [Patch(color=color, label=label[1])
           for color, label in det_habitat_scheme.items()]

ax.legend(handles=patches,
          bbox_to_anchor=(1.7, 1.0),
          facecolor="white")

# ax.set_axis_off()
plt.show()


In [ ]:
habitat_fig.savefig('outputs/Detailed_habitats_'+area_name+'_'+year+'.png')

In [ ]:
stat_summary(habitat_dataset_masked.detailed, det_habitat_scheme)

In [ ]:
# Level3plus colour scheme
waterper_scheme = {"#FFFFFF": [0., "Not affected"],
                   "#0a549e": [1., "9+ months"], 
                   "#2172b6": [2., "8 months"],
                   "#3e8ec4": [3., "7 months"],
                   "#60a6d2": [4., "6 months"],
                   "#89bfdd": [5., "5 months"],
                   "#b0d2e8": [6., "4 months"],
                   "#cde0f2": [7., "3 months"],
                   "#cde0f2": [8., "2 months"],
                   "#e8f2fb": [9., "1 month"],}

# Water/wetness persistence colour scheme
waterper_cmap = ListedColormap(list(waterper_scheme.keys()))
# Habitat classes
waterper_norm = colors.BoundaryNorm([value[0] for value in waterper_scheme.values()], 11)

In [ ]:
# Plotting
waterper_fig, ax = plt.subplots(figsize=(20, 10))

waterper_plot = ax.imshow(lc_dataset_masked.waterpersist.isel(time=0),
                     cmap=waterper_cmap,
                     norm=waterper_norm,
                     extent=[lc_dataset.x.min().data, lc_dataset.x.max().data,
                             lc_dataset.y.min().data, lc_dataset.y.max().data])

if fields_exists:
    Fields.boundary.plot(ax=ax, ec='#220f46', linewidth=1)
if boundary_exists:
    Boundary.boundary.plot(ax=ax, ec='#e72323', linewidth=3)

patches = [Patch(color=color, label=label[1])
           for color, label in waterper_scheme.items()]

ax.legend(handles=patches,
          bbox_to_anchor=(1.35, 1),
          facecolor="white")
ax.set_axis_off()
plt.show()

In [ ]:
waterper_fig.savefig('outputs/Soil_moisture_persistence_'+area_name+'_'+year+'.png')

#### Estimate how wet your land is during the year

In [ ]:
stat_summary(lc_dataset_masked.waterpersist, waterper_scheme)